<a name = 'top'></a>
# Hash the Trash notebook

Here everything should run smoothly with just this notebook, running Ganache (with quickstart) and setting the PORT on which Ganache is running (The contract's abi and bytecode has been compiled from Remix and imported here to run everything).

To see the real time version of the blockchain event log catcher: 
1. Deploy the contract from this notebook [from cell 1 through 6]
2. Launch *events_real_time.py* from the terminal and just continue through the notebook (basic prints will appear in the terminal when trashbags lifecycle events get called and they will be saved in events_log.csv)

The various sections of the notebook are:
* [Functions](#func)
* [Deploying the contract](#deploy)
* [Operating with contract's functions and eth accounts](#functions)
    * [TARI](#tari)
    * [Trash lifecycle](#cycle)
    * [Refunds](#refund)
* [Picking up logs of certain events from the chain](#logs)

In [1]:
#!pip install -r requirements.txt # if not already run

In [2]:
# imports
import random, time, json
import pandas as pd
from web3 import Web3
from contracts.abi_bytecode import abi, bytecode # saved externally as .py

# to webscrape ETH exchange rate
from bs4 import BeautifulSoup
import requests
from forex_python.converter import CurrencyRates

In [3]:
# Connecting to ganache through opened up PORT
ganache_url = 'HTTP://127.0.0.1:7545'      #change here if different
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.isConnected()

True

In [4]:
# Compiled abi and bytecode of trash.sol which inherits from agents.sol (plus Ownable and Safemath)
abiRemix = json.loads(abi)         # turned to string after copy to clipboard from Remix
bytecodeRemix = bytecode['object'] # it is a dictionary (as copy to clipboard form remix), we use the object for web3 deployment

For the purposes of the simulation, we created an Excel file containing relevant information about the actors involved in the trash chain. In particular, the Excel file *example_data.xlsx* is composed of four different sheets:
* **agents_data**: database containing information about the municipality, citizens, trucks and stations at the beginning of the year. In particular, we assumed that the municipality of Codogno has five citizens, two trucks and two stations.
* **bags_data**: simulation of trash bags generated by citizens during the year. In a real life application of this project, such data would be collected by sensors placed on trucks.
* **gps_data**: gps coordinates of the trucks when they stop to drop the garbage collected throughout the day
* **stations_data**: data collected from the station

<a name ='func'></a>
### Functions used in the notebook

These functions have been used throughout notebook and have been grouped here for easier reading.

In [5]:
def deploy_contract(deployer, _abi, _bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix (aside from getting the compiled abi and bytecode)
    
    With this function, we are able to deploy the contract just from the compiled abi and bytecode
    obtained from Remix. It would be possibile to compile the contract from Python as well 
    using *solc* library (Solidity compiler), but it is much more complicated than 
    copying the compiled abi and bytecode from Remix.
    
    Parameters
    -------------
        deployer: eth account
        abi, bytecode : compiled contract things
        
    Returns
    -------------
        contract instance
    """
    contract = web3.eth.contract(abi=_abi, bytecode=_bytecode)      # compiled contract
    tx_hash = contract.constructor().transact({'from': deployer}) # contract constructor call (i.e. deploy)
    
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)      # Get receipt when deployed to blockchain
    print(f"Deployed contract at address:\n{tx_receipt.contractAddress}") # contract address
    
    # simple yet effective method to pass the contract address to the real time filtering
    with open('data/ctr_addr.txt', 'w') as f:
        f.write(tx_receipt.contractAddress)
    
    deployed_ctr = web3.eth.contract(address = tx_receipt.contractAddress, abi  = _abi) # contract
    
    return deployed_ctr



def create(municip_addr ,data):
    '''
    Call contract functions to assign addresses (and their characteristics) roles and populate mappings
    
    Parameters
    ----------------------
        municip_addr : eth address of the municipality
        c, t, s : list of tuples with correct characteristics
        
    Returns
    ----------------------
        nothing on python, the chain grows as these transactions are registered
    '''
    # Creating list of touples as correct inputs for the contract structs

    # Create CITIZENS - (address payable _address, string memory _name, uint _family, uint _house, uint256 _w)
    c = [[r.address, " ".join([r.name, r.surname]), int(r.family), int(r.mq), int(r.weight)] 
                 for r in data.itertuples() if r.role == 'citizen']

    # Create TRUCKS - (address _address, bool _recycle)
    t = [[r.address, r.recycle] for r in data.itertuples() if r.role == 'truck']

    # Create STATIONS - (address _address, bool _recycle, int _lat, int _long)
    s = [[r.address, r.recycle, int(r.lat), int(r.long)] for r in data.itertuples() if r.role == 'station']

    from_dict = {'from': municip_addr}
    for i in range(len(c)):
        contract.functions.createCitizen(c[i][0], c[i][1], c[i][2], c[i][3], c[i][4]).transact(from_dict)
    for j in range(len(t)):
        contract.functions.createTruck(t[j][0], t[j][1]).transact(from_dict)
    for k in range(len(s)):
        contract.functions.createStation(s[k][0], s[k][1], s[k][2], s[k][3]).transact(from_dict)
    print('Roles assigned')
    
    

# Some functions to show structs statuses as we change them
def get_citizen_info(address):
    """
    Call to the contract to pretty print informations on citizen
    """
    info = contract.functions.citizens(address).call()
    print(f"""
    Address eth : {address}
    Name : {info[0]}
    Family members : {info[1]}
    House sq.meters : {info[2]}
    Assigned weight\liters : {info[3]}
    TARI amount : {info[4]}
    Recyclable Waste Tot : {info[5]}
    Non Recyclable Waste Tot : {info[6]}
    Paid TARI : {info[7]}
    Active account : {info[8]}
    """)

def get_truck_info(address):
    """Show pretty info on trucks"""
    info = contract.functions.trucks(address).call()
    print(f"""
    Address eth : {address}
    Truck number : {info[0]}
    Weight transported : {info[1]}
    Recyclable Truck : {info[2]}
    Active Truck : {info[3]}
    """)
    
    
def get_station_info(address):
    """Show pretty info for stations"""
    info = contract.functions.stations(address).call()
    print(f"""
    Address eth : {address}
    Station nr. : {info[0]}
    Weight : {info[1]}
    latitude : {info[2]}
    longitude : {info[3]}
    Recyclable Plant : {info[4]}
    Active Plant : {info[5]}
    """)
    
    
    
def get_past_logs(filter_list):
    """
    Iterates over every filter built and extracts logs from the block specified in the filter to the 'latest'
    (function called in the last section)
    
    inputs
    ---------------
     filter_list : filters created
    
    returns
    ---------------
     list containing every event attribute generated from the 'emit' on the contract
    """
    events = []
    for event_filter in filter_list:
        for e in event_filter.get_all_entries(): # get_new_entry() to check only last block
            
            # e is a nested dictionary, like this we bring all elements on the same level
            args = dict(e['args'])
            args.update(dict(e))
            del args['args'] # brought one level above, delete the nest
            # args.pop('args', None) # could delete like this too
            
            events.append(args)
            
    return events

<a name = 'deploy'></a>

### Deploying the contract

At the beginning of each year, the Municipality (in our example, the Municipality of Codogno) has to deploy the smart contract and invoke the function *setBeginningYear* to set the varible *start* equal to 1st January. This will come useful for the Municipality when perfoming some payment checks throughout the year. However, for the purposes of this simulation, we have decided to not include all the time contraints set in the smart contracts. In this way, we can indeed call all the functions without incurring in any problem. For example, we don't have to wait almost one year before being able to compute and pay the payouts to the citizens. 

In [6]:
# Simple database example. The column with ganache accounts is added to later interact with them
data = pd.read_excel('data/example_data.xlsx', sheet_name='agents_data', engine='openpyxl')
data['address'] = web3.eth.accounts     
data

,name,surname,family,mq,weight,recycle,lat,long,role,address
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0xc67d4642cFDceFAa5Bd0a9e206c32A6b67Faa79D
1,Francesca,Bianchessi,2.0,70.0,500.0,NaN,NaN,NaN,citizen,0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
2,Ilaria,Bolla,4.0,200.0,900.0,NaN,NaN,NaN,citizen,0x9ff8d68b956f11D3f4ba22274214436F22cF878C
3,Alessandro,Botti,5.0,340.0,800.0,NaN,NaN,NaN,citizen,0xCb75625D2bd0b014071aF1690C6f8F49825b25Fd
4,Davide,Castellini,3.0,50.0,200.0,NaN,NaN,NaN,citizen,0x0c3DF77F68fcC27BA8e3ebC35EdcE51779efA831
5,Anna,Di Marco,4.0,80.0,560.0,NaN,NaN,NaN,citizen,0x493aD74df90b487E138DaF6454B28d879E561da2
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0x820f19622A84553C5EEfeC59c155A1bCf1097b6f
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x799AfB7AEd85ABA3dB8ee78d192Db6A8c5c6C442
8,Disposal Station,Milano,NaN,NaN,NaN,True,45.4642,9.1900,station,0xb4E3D19ef8113D78fd72b7568756E11CC25510bd
9,Disposal Station,Bologna,NaN,NaN,NaN,False,44.4949,11.3426,station,0xf4CD18B9596Ab6219A0253cccCC12A444FB0E1Dc


In [7]:
# Deploy contract (the municipality is the one deplying in our case). You can check the blocks from Ganache to see
# if it has worked.
municipality = data[data.role == 'municipality']['address'].item()

contract = deploy_contract(deployer = municipality, _abi = abiRemix, _bytecode = bytecodeRemix)

Deployed contract at address:
0x29793f9B5959308F35383ac999843ec1b8E166a8


##### !! You can now run `events_real_time.py` !!

<a name = 'functions'></a>
## Interacting with the functions

We first perform a simple check to verify whether the actual owner of the contract is the municipality. We then create the agents with the functions defined in the smart contract.

**Reminder:** function(...).**transact**(...) is used for calls to functions that modify the chain, otherwise use function(...).**call**(...) (e.g. for view functions)

In [8]:
# simple check for owner of the contract
owner = contract.functions.owner().call() # get owner from contract function
print(f"owner: {owner}")
print(f"Is it the municipality? {owner == data[data.role == 'municipality']['address'].item()}")

# to remember inputs and all functions
contract.all_functions()

owner: 0xc67d4642cFDceFAa5Bd0a9e206c32A6b67Faa79D
Is it the municipality? True


[<Function MunicipalityBalance()>,
 <Function TariAmount(address)>,
 <Function citizens(address)>,
 <Function createCitizen(address,string,uint256,uint256,uint256)>,
 <Function createStation(address,bool,int256,int256)>,
 <Function createTruck(address,bool)>,
 <Function deleteCitizen(address)>,
 <Function deleteStation(address)>,
 <Function deleteTruck(address)>,
 <Function destroyContract()>,
 <Function drop(address,int256,int256)>,
 <Function givePayout(address)>,
 <Function numberC()>,
 <Function numberS()>,
 <Function numberT()>,
 <Function owner()>,
 <Function payTari()>,
 <Function pick(address,uint256,uint256)>,
 <Function received(bool,address,uint256)>,
 <Function renounceOwnership()>,
 <Function setBeginningYear()>,
 <Function showStartTime()>,
 <Function stations(address)>,
 <Function transferOwnership(address)>,
 <Function trucks(address)>,
 <Function withdraw()>]

In [9]:
# Populating mappings and checking amount of entities
create(municipality, data)

print(f"n° Citizens: {contract.functions.numberC().call()}, n° Trucks : {contract.functions.numberT().call()}, n° Stations : {contract.functions.numberS().call()}")

Roles assigned
n° Citizens: 5, n° Trucks : 2, n° Stations : 2


In [10]:
# Using exemplar citizen and other roles to do checks along the notebook
dutiful_citizen = data[data.role == 'citizen']['address'].reset_index(drop = True)[0]
get_citizen_info(dutiful_citizen)

ex_truck = data[data.role == 'truck']['address'].reset_index(drop = True)[0]
get_truck_info(ex_truck)

ex_station = data[data.role == 'station']['address'].reset_index(drop = True)[0]
get_station_info(ex_station)


    Address eth : 0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 0
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    

    Address eth : 0x820f19622A84553C5EEfeC59c155A1bCf1097b6f
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    

    Address eth : 0xb4E3D19ef8113D78fd72b7568756E11CC25510bd
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


<a name = 'tari'></a>
### Compute TARI 

The TARI is here computed by summing two parts:
1. The first part is given by the product of the square meters of the property and a fixed fee which depends on the number of people in the  household. If there are less than four people in the household, the fee *deposit_mq_less4 defined* in the smart contract applies, whereas if the household has more than four members, the fee *deposit_mq_more4* applies.
2. The second part is variable and depends on the total amount of waste produced by the household the year before. To be more specific, the total weight of waste produced by the household the year before is multiplied by a constant amount of money.

At the beginning of each year, the Municipality has to call the *TariAmount* function to compute how much TARI each citizen has to pay. The Municipality is then in charge of informing each citizen (e.g. through a text message or an app notification).

In [11]:
# Calculate amount of TARI for each citizen (and looping for the others)
for addr in data[data.role == 'citizen']['address']:
    contract.functions.TariAmount(addr).transact({'from' : municipality})

get_citizen_info(dutiful_citizen)


    Address eth : 0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    


The TARI amount has been calculated and assigned. In particular, the TARI is expressed in wei for each citizen. We can thus now compute its equivalent in euros and save this information.

In [12]:
# Webscrape currency exchange rates to convert computed TARI in wei to euro
# Save the TARI amount for each citizen 
tari_list = [contract.functions.citizens(addr).call()[4]*(10**(-18)) 
             for addr in data[data.role == 'citizen']['address']]
data.loc[data.role == 'citizen', 'TARI_eth'] = tari_list

# Convert to EUR 
cmc = requests.get('https://coinmarketcap.com/currencies/ethereum/markets/')
soup = BeautifulSoup(cmc.content, 'html.parser')  
data_coinmkt = soup.find('script', type="application/ld+json")
data_coinmkt = json.loads(data_coinmkt.contents[0])
usd_eth = data_coinmkt['currentExchangeRate']['price']

c = CurrencyRates()
usd_eur_rate = c.get_rate('USD', 'EUR') 
eur_eth = usd_eth * usd_eur_rate 

data.loc[data.role == 'citizen', 'TARI_eur'] = data.loc[data.role == 'citizen', 'TARI_eth']*eur_eth 
data[['name', 'surname', 'family', 'mq', 'weight', 'role', 'TARI_eth', 'TARI_eur']]

,name,surname,family,mq,weight,role,TARI_eth,TARI_eur
0,Comune,Codogno,0.0,0.0,0.0,municipality,NaN,NaN
1,Francesca,Bianchessi,2.0,70.0,500.0,citizen,0.190,93.567206
2,Ilaria,Bolla,4.0,200.0,900.0,citizen,0.490,241.304899
3,Alessandro,Botti,5.0,340.0,800.0,citizen,1.440,709.140927
4,Davide,Castellini,3.0,50.0,200.0,citizen,0.120,59.095077
5,Anna,Di Marco,4.0,80.0,560.0,citizen,0.216,106.371139
6,Truck,Recycle,NaN,NaN,NaN,truck,NaN,NaN
7,Truck,Not Recycle,NaN,NaN,NaN,truck,NaN,NaN
8,Disposal Station,Milano,NaN,NaN,NaN,station,NaN,NaN
9,Disposal Station,Bologna,NaN,NaN,NaN,station,NaN,NaN


In [13]:
# Each citizen pays its due amount
for addr in data[data.role == 'citizen']['address']:
    
    info = contract.functions.citizens(addr).call() # need the amount from the contract
    print(f"Citizen {info[0]} is paying {info[4]} wei --> {round(info[4]*10**(-18)*eur_eth, 2)} Euro")
    
    # payable function, the amount is passed in the dictionary of .transact()
    contract.functions.payTari().transact({'from': addr, 'value' : info[4]})
    

Citizen Francesca Bianchessi is paying 190000000000000000 wei --> 93.57 Euro
Citizen Ilaria Bolla is paying 490000000000000000 wei --> 241.3 Euro
Citizen Alessandro Botti is paying 1440000000000000000 wei --> 709.14 Euro
Citizen Davide Castellini is paying 120000000000000000 wei --> 59.1 Euro
Citizen Anna Di Marco is paying 216000000000000000 wei --> 106.37 Euro


At this point, all the citizens in the simulation have paid the TARI. To check whether this is actually true, let us print the information stored in the struct of a citizen (i.e. dutiful_citizen). As you can see, the attribute "Paid TARI" is now equal to true, signalling that the citizen Francesca has respected the law and paid the due amount of TARI.

In [14]:
get_citizen_info(dutiful_citizen)


    Address eth : 0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : True
    Active account : True
    


All the citizens should pay the TARI by the end of January. It would also be possible to add some functionalities to deal with the case of non-compliance with this rule. For example, if a citizen didn't pay the TARI on time, the Municipality could solicit the payment through, for example, a text message or an app notification, and maybe also decrease the reimbursement by a certain amount at the end of the year.

<a name = 'cycle'></a>
## Working with trashbags lifecycles

At this point of the simulation, we use as input the data collected in the Excel sheet *bags_data*, which contains information about the single trash bags generated by the citizens. The trucks will pick such trash bags up and drop them at the appropriate disposal station. In a real life application of this project, the data from the Excel sheet would be collected by sensors placed on trucks.

In [15]:
bags = pd.read_excel('data/example_data.xlsx', sheet_name='bags_data', engine='openpyxl')
bags.head(6) 

,name,surname,weight,recycle
0,Francesca,Bianchessi,1,True
1,Ilaria,Bolla,2,False
2,Alessandro,Botti,3,True
3,Davide,Castellini,1,False
4,Anna,Di Marco,2,True
5,Francesca,Bianchessi,1,True


### Pick up trash

The function for the pick up of trash bags is called by the truck. Each truck is equipped with some sensors, which scan and store the information printed on each trash bag (i.e. the Ethereum address of the citizen who has generated a specific trash bag), and with a scale, which is used to determine the weight of each trash bag.

In [16]:
# The right truck will pick up the various trashbags
for i, name, sur, w, recyclable in bags.itertuples():

    # get address of generator
    generator_addr = data[data.name == name]['address'].item()
    
    # get correct truck address via subsetting
    truck_addr = data[(data['name'] == 'Truck') & (data['recycle'] == recyclable)]['address'].item()
    
    # 'pick' function in contract is called by the correct truck
    contract.functions.pick(generator_addr, w, random.randint(0, 1e+10)).transact({'from' : truck_addr})
    
get_citizen_info(dutiful_citizen)
get_truck_info(ex_truck)


    Address eth : 0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 16
    Non Recyclable Waste Tot : 12
    Paid TARI : True
    Active account : True
    

    Address eth : 0x820f19622A84553C5EEfeC59c155A1bCf1097b6f
    Truck number : 1
    Weight transported : 71
    Recyclable Truck : True
    Active Truck : True
    


As you can see, waste counters increased for both the citizen (have a look at the attributes *Recyclable Waste Tot*
and *Non Recyclable Waste Tot*) and for the truck (have a look at the attribute *Weight transported*)

### Drop bags at station

The function for the dumping of trash bags at the appropriate disposal station is still called by the trucks. Once a truck has dropped its content at the station, it will result empty, while the station will instead increase its total counter with the received weight.

The **drop** function requires as input the GPS coordinates of the truck when it gets to the station. For the purposes of this simulation, we are going to extract this information from the Excel sheet *gps_data*.

In [17]:
gps = pd.read_excel('data/example_data.xlsx', sheet_name='gps_data', engine='openpyxl')
gps

,name,surname,lat,long
0,Disposal Station,Milano,45.4642,9.1900
1,Disposal Station,Bologna,44.4949,11.3426


In [18]:
get_station_info(ex_station)


    Address eth : 0xb4E3D19ef8113D78fd72b7568756E11CC25510bd
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


In [19]:
# get station name and coords from the gps
for i, name, sur, lat, long in gps.itertuples():
    
    #get station address
    station_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    #get_station_info(station_addr)
    # need to get the type of station (recycling/not recycling) to pair the truck
    s_info = contract.functions.stations(station_addr).call() # s_info[4] is the type
    
    # pairing truck based on recyclable or not and calling the 'drop' function
    correct_truck = data[(data.name == 'Truck') & (data.recycle == s_info[4])]['address'].item()
    
    contract.functions.drop(station_addr, int(lat), int(long)).transact({'from' : correct_truck})

get_station_info(ex_station)
get_truck_info(ex_truck)    


    Address eth : 0xb4E3D19ef8113D78fd72b7568756E11CC25510bd
    Station nr. : 1
    Weight : 71
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    

    Address eth : 0x820f19622A84553C5EEfeC59c155A1bCf1097b6f
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    


The station's total weight has now increased by an amount equal to the weight of trash carried by the truck. On the other hand, the truck is now empty (the attribute "weight transported is now equal to zero").

### Received 

The **received** function can only be called by the station, and its purpose is to verify whether there is coherence between the total amount of trash that a station declares to have received up to now, and the amount of trash that has been actually dumped to the station by trucks over time. When calling this function, the station has to specify the Ethereum address of the truck that has just arrived to the station (the address is read by some sensors placed at the station), the type of waste that the station disposes and the total weight of trash that has been dropped at the station so far (thanks to the help of some scales).

In [20]:
station_sense = pd.read_excel('data/example_data.xlsx', sheet_name='stations_data', engine='openpyxl')
station_sense

,name,surname,weight
0,Truck,Recycle,71
1,Truck,Not Recycle,69


In [21]:
for i, name, sur ,weight in station_sense.itertuples():
    
    # get truck address
    truck_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    # correct station to read the correct type of truck [otherwise we get an error]
    info_tr = contract.functions.trucks(truck_addr).call() # to get truck type in info_tr[2] 
    st_addr = data[(data.name == 'Disposal Station') & (data.recycle == info_tr[2])]['address'].item() 
    print(f"Station: {st_addr}\nFrom Truck: {truck_addr}\n ")
    
    #recieved function call
    contract.functions.received(info_tr[2], truck_addr, weight).transact({'from' : st_addr})
    

Station: 0xb4E3D19ef8113D78fd72b7568756E11CC25510bd
From Truck: 0x820f19622A84553C5EEfeC59c155A1bCf1097b6f
 
Station: 0xf4CD18B9596Ab6219A0253cccCC12A444FB0E1Dc
From Truck: 0x799AfB7AEd85ABA3dB8ee78d192Db6A8c5c6C442
 


<a name = 'refund'></a>
## Refund

At the end of each year, the municipality calls the "givePayout" function and reimburses the citizens on the basis of their recycling behaviours. In particular, the municipality is allowed to call this function only between 20th and 28th December (considering that the municipality has deployed the contract on 1st January). However, for the purposes of this simulation, we have commented the line of the code that performes this "time check", so that we could actually call the "givePayout" function without incurring in an error.

In [22]:
# citizen address as input to function
for citizen in data[data.role == 'citizen']['address']:
    
    # balance comparison for the refund
    before = web3.eth.getBalance(citizen)
    contract.functions.givePayout(citizen).transact({'from' : municipality})
    after = web3.eth.getBalance(citizen)
    
    print(f"{citizen}\nRefunded wei :{after-before} --> Euro : {round((after-before)*10**(-18)*eur_eth, 2)}\n")
        

0x21B06246a8c3a977457Fe7F86655F3E1270f20BC
Refunded wei :9500000000000000 --> Euro : 4.68

0x9ff8d68b956f11D3f4ba22274214436F22cF878C
Refunded wei :9800000000000000 --> Euro : 4.83

0xCb75625D2bd0b014071aF1690C6f8F49825b25Fd
Refunded wei :72000000000000000 --> Euro : 35.46

0x0c3DF77F68fcC27BA8e3ebC35EdcE51779efA831
Refunded wei :2400000000000000 --> Euro : 1.18

0x493aD74df90b487E138DaF6454B28d879E561da2
Refunded wei :10800000000000000 --> Euro : 5.32



The municipality has the possibility to withdraw some funds from the contract before the end of the year, and thus enjoy some immediate liquidity. However, the municipality is only allowed to withdraw at most 88% of the money in the contract to not compromise the ability of reimbursing all the citizens at the end of the year.

In [23]:
#print(municipality) # we have the municipality address stored from before
# the withdraw works just like this. See ganache '0' account, i.e. the account of the municipality
before = web3.eth.getBalance(municipality)
contract.functions.withdraw().transact({'from' : municipality}) 
after = web3.eth.getBalance(municipality)
print(f"Withdrawn: {(after-before)/(10**18)} ETH (also minus gas costs)")

# Check how much money is still stored in the contract 
balance = contract.functions.MunicipalityBalance().call({'from' : municipality})
print('On contract there is still :', balance/(10**18), 'ETH')

Withdrawn: 2.06828146 ETH (also minus gas costs)
On contract there is still : 0.28218 ETH


At the end of each year, the municipality, after reimbursing all the citizens, calls the **destroyContract** function and receives all the funds that were still stored in the contract. 

In [24]:
# selfdestruct contract by invoking destroyContract function
contract.functions.destroyContract().transact({'from' : municipality})

HexBytes('0x5fc5a773c57d64ad7a03d14560a5344ab4dd94814bd2ca9c96e14eaa5a148655')

<a name = 'logs'></a>

## Event/Log Filtering

After running the above cells, the blockchain is populated with a lot of different logs from events. Such logs can be extracted with specific filters that are contract specific. 

For the sake of the demonstration, we employ just one filter and then show an example of the database that is created. A ***real time*** version that repeatedly checks the last block can instead be found in the file *events_real_time.py*.

In [25]:
# creating logs/event filters to inspect the whole chain (fromBlock = 1)

# filter for the Event 'PickedUp' generated by the truck
pickedUp_filter = contract.events.PickedUp.createFilter(fromBlock = 1)
#recieved_filter etc

filters = [pickedUp_filter]

# Pass filters on the chain (from block 1) and pick logs found (for now only 'ToPickUp')
events = get_past_logs(filters)
df = pd.DataFrame(events)
df.head()

,transporter,wasteType,bagId,generator,wasteWeight,pickUpTime,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber
0,0x820f19622A84553C5EEfeC59c155A1bCf1097b6f,True,"b'\x1c\xdf\xde\xa3\xd6Y""x\x85m\xc8\xe8u\xf5\xf...",0x21B06246a8c3a977457Fe7F86655F3E1270f20BC,1,1608122375,PickedUp,0,0,b'\xf1\x8c\xefy\x17\x83\x18\x90\xd1\x19\xdd\x0...,0x29793f9B5959308F35383ac999843ec1b8E166a8,"b""p2\x01\xff\x8d\x10`:\xb6\xc3\x96\x1eV'U\x17\...",85
1,0x799AfB7AEd85ABA3dB8ee78d192Db6A8c5c6C442,False,b'\x10eLl\x84\no\xf2{\xc5\x8b\x0f\xbf\x0cM\x90...,0x9ff8d68b956f11D3f4ba22274214436F22cF878C,2,1608122376,PickedUp,0,0,b'\x8eru\xc7\xfdx?\xa5L{\x1e/\xe6\xd6ig\xf6\r$...,0x29793f9B5959308F35383ac999843ec1b8E166a8,b'\x87\xc1T\xd8\x0b\x1dnb\xa1\xe6|\x03E\xe7jl\...,86
2,0x820f19622A84553C5EEfeC59c155A1bCf1097b6f,True,b'\xbf\x8fs\xac\x82\xa6j\x8d\x95W\xf0SSe\xb8\x...,0xCb75625D2bd0b014071aF1690C6f8F49825b25Fd,3,1608122376,PickedUp,0,0,b'Q\x91v\xfe\x8b\x8d|a\xaa\xae8/\xb8\xd8~\xe5+...,0x29793f9B5959308F35383ac999843ec1b8E166a8,b'\xc5eN\x8b~4py\x82~\xf1\x18`\xd1NR\xday\xe9\...,87
3,0x799AfB7AEd85ABA3dB8ee78d192Db6A8c5c6C442,False,b'\xde\xb6\xb9\xed\xc6_\xc9$Y#\xab\xaa\xd8\xb4...,0x0c3DF77F68fcC27BA8e3ebC35EdcE51779efA831,1,1608122377,PickedUp,0,0,b'q\x93\xd1\n\x80\xfa\xf4\x1c\xde\xc7\xa6}r\x8...,0x29793f9B5959308F35383ac999843ec1b8E166a8,"b'sU\xf7I\xa0\x9a\x8eK\xa9\xfc\x0f\x94j""\x00&\...",88
4,0x820f19622A84553C5EEfeC59c155A1bCf1097b6f,True,b'\xef|:\x80o)\xd4\xb5y\x98\xfe\xfeT\xbc\t\xc7...,0x493aD74df90b487E138DaF6454B28d879E561da2,2,1608122378,PickedUp,0,0,b'e\xb0\xbbW`\xc0\xb2n\x83\x95Y\x00\xb0\xb1\xc...,0x29793f9B5959308F35383ac999843ec1b8E166a8,b'\xff#y\xa8\x9f\xb7\xd0?\x99\x03R\xbf\xba\x0e...,89


Each line of the dataframe represents a trash bag being picked up by a truck. In particular, for each trash bag being collected, you can get information about:
* **transporter**: the Ethereum address of the transporter
* **wasteType**: whether the trash bag containes recyclable or non-recyclable waste
* **bagId**: the unique id representing the trash bag
* **generator**: the Ethereum address of the citizen who has generated the trash bag
* **wasteWeight**: the weight of the trash bag
* **pickUpTime**: the time when the trash bag was picked up by the truck
* **event**: the type of event. In this case, we are only focusing on the event "PickedUP"
* **logIndex**: the log index
* **transactionIndex**: the transaction index
* **transactionHash**: the hash of the transaction
* **address**: the address of the deployed contract
* **blockHash**: the hash of the block
* **blockNumber**: the number of the block

(The first six columns are characterizing the *PickedUp* event during the emit, for other events different columns are present)

[Back to top](#top)